### 非线性支持向量机
**高斯核函数**
**smo算法**

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

#    导入sklearn自带的数据集
from sklearn.datasets import load_iris


In [4]:
iris = load_iris()
#    转为DataFrame
irisData = pd.DataFrame(iris.data, columns=iris.feature_names)
irisData['Label'] = iris.target

#    只用两个分类，因此将Label=2的删除
irisData = irisData[irisData['Label'] < 2]
#    将数据转为矩阵形式，并区分feature和label
X = np.array(irisData.iloc[:,:-1])
y = np.array(irisData['Label'])
#    感知机模型的输入label应为 -1 ，1
y[y==0] = -1

X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=5)


#cancer = load_breast_cancer()
#X = cancer.data
#y = cancer.target
#
##label集合应为 -1 ， 1
#y[y==0] = -1
##    分割为训练集、测试集
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

高斯核函数

In [31]:
def gaussian_kernel(x, z):
    '''
    高斯核函数
    '''
    return sum([x[k] * z[k] for k in range(len(x))])

序列最小最优化(SMO)算法

In [32]:
def SMO(X_train, y_train, kexin):
    '''
    序列最小最优化算法
    X_train, y_train: 训练数据集
    kexin: 精度
    '''
    pass
    

In [38]:
b = 0
C = 1

#    初始化α
alpha = np.zeros(len(X_train))
N = len(X_train)
def gx(xi):
    
    temp = 0
    for j in range(N):
        temp += alpha[j]*y[j]*gaussian_kernel(xi, X_train[j])
    return temp + b

        

In [39]:
Ei = []

for i in range(N):
    
    ei = gx(X_train[i]) - y_train[i]
    Ei.append(ei)

In [40]:
def KKT(alpha_i):
    '''
    检验训练样本点(xi,yi)是否满足KKT条件
    '''
    y_g = y_train[alpha_i[1]]*gx(X_train[alpha_i[1]])
    if alpha_i[0] == 0:
        return y_g >= 1 
    elif alpha_i[0] > 0 and alpha_i[0] < C:
        return y_g == 1
    elif alpha_i[0] == C:
        return y_g <= 1 
        

In [43]:
def optimization(alpha1, alpha2):
    
    #    最优化
    if y_train[alpha2[1]] != y_train[alpha1[1]]:
        L = max(0, alpha2[0] - alpha1[0])
        H = min(C, C + alpha2[0] - alpha1[0])
    else:
        L = max(0, alpha2[0] + alpha1[0] - C)
        H = min(C, alpha2[0] + alpha1[0])
    
    
    alpha2_unc = alpha2
    yita = gaussian_kernel(X_train[alpha1[1]], X_train[alpha1[1]]) +gaussian_kernel(X_train[alpha2[1]], X_train[alpha2[1]]) - 2*gaussian_kernel(X_train[alpha1[1]], X_train[alpha2[1]])
    alpha2_unc[0] = alpha2[0] + (y_train[alpha2[1]]*(E1-E2)/yita)
    alpha2_new = alpha2_unc
    if alpha2_unc[0] > H:
        alpha2_new[0] = H
    elif alpha2_unc[0] < L:
        alpha2_new[0] = L
    alpha1_new = alpha1
    alpha1_new[0] = alpha1[0] + y_train[alpha1[1]]*y_train[alpha2[1]]*(alpha2_new[0] - alpha2[0])
    #    更新α
    alpha[alpha1_new[1]] = alpha1_new[0]
    alpha[alpha2_new[1]] = alpha2_new[0]       
   
    return alpha1_new, alpha2_new

In [44]:

iter = 200
while iter != 150:
    iter -= 1
    #    选择待优化的两个变量
            
    #    首先在支持向量中寻找不满足KKT条件的样本点
    alpha1_init = [i for i in range(N) if 0<alpha[i]<C]
    #    若没有，则遍历整个训练数据集
    alpha1_init_notsv = [i for i in range(N) if i not in alpha1_init]
    alpha1_init.extend(alpha1_init_notsv)
    
    Ei_min = [float('inf'), -1]
    Ei_max = [float('-inf'), -1]
    for i in range(N):
        ei = Ei[i]
        if ei < Ei_min[0]:
            Ei_min = [ei, i]
        if ei > Ei_max[0]:
            Ei_max = [ei, i]
   
    for i in range(len(alpha1_init)):
        alpha1 = [alpha[alpha1_init[i]], alpha1_init[i]]
        if not KKT(alpha1):
            E1 = Ei[alpha1[1]]
            #    第二个变量的选择
            if E1 < 0:
                alpha2 = Ei_max
            else:
                alpha2 = Ei_min
            E2 = alpha2[0]
            alpha2[0] = alpha[alpha2[1]]
            #    最优化
            alpha1_new, alpha2_new = optimization(alpha1, alpha2)
            #    更新b
            b = -E1 - y_train[alpha1[1]]*gaussian_kernel(X_train[alpha1[1]], X_train[alpha1[1]])*(alpha1_new[0] - alpha1[0]) - y_train[alpha2[1]]*gaussian_kernel(X_train[alpha2[1]], X_train[alpha1[1]])*(alpha2_new[0] - alpha2[0]) + b
            #    更新Ei
            Ei_new = [0]*N
            for j in range(N):
                for k in range(len(alpha1_init)):
                    Ei_new[j] += y_train[alpha1_init[k]]*alpha[alpha1_init[k]]*gaussian_kernel(X_train[j], X_train[alpha1_init[k]])
            
                Ei_new[j] = Ei_new[j] + b - y_train[j]
                
            Ei = Ei_new
            print(b)
            break

alpha

-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908
-15.688486265341908


array([0.        , 0.        , 0.11689071, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.11689071, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])